In [16]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Pandas is a data manipulation library 
import pandas as pd


In [2]:
# load the dataset
dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [3]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [4]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)

print('Accuracy: %.2f ' % (accuracy*100))

In [ ]:
Xnew = [[1,85,66,29,0,26.6,0.351,31]]
#// 7,196,90,0,0,39.8,0.451,41 => TRUE
# // 1,85,66,29,0,26.6,0.351,31	=> FALSE
predictions = model.predict_classes(Xnew)

print("Resultat: %.2f " %  predictions)



In [9]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
# Save the model to disk
model.save(MODEL_TF)

In [ ]:
# Convert the model to the TensorFlow Lite format 
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_no_quant_tflite)



In [14]:
# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_tflite = os.path.getsize(MODEL_TFLITE)

In [ ]:
# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_tflite} bytes ", f"(reduced by {size_tf - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}


In [ ]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}